# MongoDB Handling

After installing the MongoDB server in your machine, you can use this notebook for handling the initial processes with the database.

Specifically, in this step, we utilize Python's `pymongo` library to exploit its capabilities for MongoDB server interaction.

**Important Note: Be sure that the MongoDB server is up and running as a service in the background.**

For example, in macOS, to run MongoDB (i.e. the mongod process) as a service, run:

* `brew services start mongodb-community`

To stop a mongod running as a macOS service, use the following command as needed:

* `brew services stop mongodb-community`

To install MongoDB in your system, follow the instructions here:

* https://www.mongodb.com/docs/manual/administration/install-community/


**Note:** You can modify any of the processes below, however, you have to explain your thoughts.

In [3]:
# import library for various processes with the OS
import os

## Load configuration

In [4]:
# import library for yaml handling
import yaml

In [5]:
config_path = os.path.join(os.getcwd(), "config.yml")

with open(config_path) as file:
    config = yaml.load(file, Loader=yaml.FullLoader)

## MongoDB database instantiation

The relevant information for the MongoDB client connection, the database name, and collection name is located in the configuration file.

```
# DB Connection with the uri (host)
client: "mongodb://mymongo:27017"

# db name
db: "aiot_course"

# db collection
col: "gesture_data"
```

In [6]:
# import library for hanlding the MongoDB client
import pymongo
# import library for retrieving datetime
from datetime import datetime

### Create the database

To create a database in MongoDB, start by creating a MongoClient object, then specify a connection URL with the correct ip address and the name of the database you want to create.

MongoDB will create the database if it does not exist, and make a connection to it.

In [7]:
client = pymongo.MongoClient(config["client"])

In [8]:
db = client[config["db"]]

### Instantiate the collection

To create a collection in MongoDB, use database object and specify the name of the collection you want to create.

MongoDB will create the collection if it does not exist.

In [9]:
col = db[config["col"]]

Initially, no collection will be shown in MongoDB before you enter the first document!

## Create the data collection

Uploading the gathered data to MongoDB collection. The data directory structure should be as follows:

```
.
└── data/
    ├── class_A/
    │   ├── data_A_01.csv
    │   ├── data_A_02.csv
    │   └── ..
    ├── class_B/
    │   ├── data_B_01.csv
    │   ├── data_B_02.csv
    │   └── .
    └── class ...
```

In [10]:
# import library for hanlding the csv data and transformations
import pandas as pd
import json

Get data path:

In [11]:
data_path = os.path.join(os.getcwd(), "data")
print(data_path)

/home/jovyan/work/data


List all files in a path:

In [12]:
classes_folders_list = [f for f in os.listdir(data_path) if os.path.isdir(os.path.join(data_path, f))]
print(classes_folders_list)

['classA', 'classL', '.ipynb_checkpoints']


In [13]:
# print files in folder
folder_path = os.path.join(data_path, classes_folders_list[0])
files_in_folder = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
print(files_in_folder)

['A_02.csv', 'A_08.csv', 'A_05.csv', 'A_04.csv', 'A_06.csv', 'A_07.csv', 'A_03.csv', 'A_09.csv', 'A_10.csv', 'A_01.csv']


Each document in the MongoDB database should have the following schema:

```json
{
  "data": {
    "acc_x": ["array", "of", "values"],
    "acc_y": ["array", "of", "values"],
    "acc_z": ["array", "of", "values"],
  },
  "label": "The label of the instance",
  "datetime": "MongoDB datetime object (it can be generated with the datetime.datetime.now() function"
}
```

Accordingly, if you are using gyroscope or both accelerometer and gyroscope, the following order and naming of the sensor keys should be defined:

* for gyroscope: `gyr_x`, `gyr_y`, `gyr_z` for the three axes
* for accelerometer and gyroscope: `acc_x`, `acc_y`, `acc_z`, `gyr_x`, `gyr_y`, `gyr_z` for the six axes

**Note: Be careful, the document is mandatory to have the aforementioned schema, in order to argue and proceed with the rest of the processes later on, in data engineering, plotting, etc.**

In [14]:
import yaml
from utils import df_rebase

## Provide the code to upload the data to MongoDB

In [17]:
try:
    # Test connection
    client.server_info()
    print("Connection succesful")

    # Show databases
    print("Databases:", client.list_database_names())

except ServerSelectionTimeoutError as e:
    print("Could not connect to MongoDB:", e)

Connection succesful
Databases: ['admin', 'aiot_course', 'config', 'local']


In [15]:
def load_and_upload(class_name, label):
    class_path = os.path.join(data_path, class_name)
    csv_files = [f for f in os.listdir(class_path) if f.endswith('.csv')]
    
    for csv_file in csv_files:
        file_path = os.path.join(class_path, csv_file)
        df = pd.read_csv(file_path)
        
        if len(df.columns) == 3:
            df.columns = ['acc_x', 'acc_y', 'acc_z']
        else:
            df = df.iloc[:, 1:4]
            df.columns = ['acc_x', 'acc_y', 'acc_z']
        
        df['label'] = label
        records = df.to_dict(orient='records')
        col.insert_many(records)
        print(f"Uploaded {len(records)} from {csv_file}")

load_and_upload("classA", "A")
load_and_upload("classL", "L")

print("Done")

Uploaded 1293 from A_02.csv
Uploaded 1194 from A_08.csv
Uploaded 1241 from A_05.csv
Uploaded 1301 from A_04.csv
Uploaded 1290 from A_06.csv
Uploaded 1396 from A_07.csv
Uploaded 1342 from A_03.csv
Uploaded 1260 from A_09.csv
Uploaded 1307 from A_10.csv
Uploaded 1380 from A_01.csv
Uploaded 1061 from L_08.csv
Uploaded 1042 from L_05.csv
Uploaded 1031 from L_02.csv
Uploaded 1101 from L_09.csv
Uploaded 1063 from L_10.csv
Uploaded 855 from L_01.csv
Uploaded 1078 from L_07.csv
Uploaded 1034 from L_03.csv
Uploaded 1023 from L_04.csv
Uploaded 1021 from L_06.csv
Done
